In [1]:
import pandas as pd
import get_all_vars
import read_vars_for_separate_models_for_dependent_vars
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import settings
from helpers import dummy_utils

bucket = S3BucketUtils()

def get_key(val, dict_):
    for key, value in dict_.items():
        if val == value:
            return key


def drop_duplicates_from_a_list_of_lists(list_of_lists):
    for i in range(0, len(list_of_lists)):
        list_of_lists[i] = sorted(list_of_lists[i])

    list_of_lists = \
    [ast.literal_eval(x) for x in list(dict.fromkeys([str(x) for x in list_of_lists]))]

    return list_of_lists


(behavioural_vars_original, behavioural_vars_base_names) = get_all_vars.main()


rest_of_the_vars = ['Website.Views.last.month.total.log2',\
                    'SpotHopper_site_not_live_yet',\
                    'their_own_website_dontKnow',\
                    'their_own_website_no',\
                    'Annual Commitment with 2 semi-annual payments',\
                    'Annual Prepay',\
                    'Monthly',\
                    'is_premium_plus',\
                    'is_platform_only',\
                    'is_corona',\
                    'Opted.out.of.facebook',\
#                     'Instagram.Add.on',\
                    'Fb.page.likes.analysis.avg.log2',\
                    'New.email.subscriber.signups.last.month.div10',\
                    'New.email.subscribers.allinclusive.last.month.div10',
                    'instagram_New', 'instagram_Old',\
                    'slider_balanced','slider_increase_revenue','slider_save_time',\
                    'payment_method_ACH', 'payment_method_CC', 'payment_method_CHECK']

### Added year_become_customer dummy as regular var in churn ###
year_become_customer = dummy_utils.get_dummies_col_names_as_intersection_of_datasets('year_become_customer')
rest_of_the_vars = rest_of_the_vars + year_become_customer

all_vars = \
pd.DataFrame(list(behavioural_vars_base_names.values())+rest_of_the_vars).rename(columns = {0:'base_var_name'})
all_variables = pd.DataFrame.from_dict(behavioural_vars_base_names, orient='index').reset_index().\
rename(columns = {'index':'original_var_name', 0:'base_var_name'})

all_variables = \
all_vars.merge(all_variables, on = ['base_var_name'], how = 'left')
bucket.store_csv_to_s3(data_frame = all_variables, \
                           file_name = 'all_variables.csv', \
                           dir = '/churn_analysis_based_on_behaviour/')